In [44]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [45]:
# Set up diretory paths
base_dir = "Combined_Dataset/"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

In [46]:
# Set up data generators
image_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
# Resize Images
def preprocess_image(image):
    image = cv2.resize(image, image_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

In [48]:
# Convert Images to Arrays
images = []
labels = []
for category in os.listdir(train_dir):
    category_dir = os.path.join(train_dir, category)
    for img_path in os.listdir(category_dir):
        img = cv2.imread(os.path.join(category_dir, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = preprocess_image(img)
        images.append(img)
        labels.append(category)

In [49]:
# One-Hot Encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [50]:
# Step 11: Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [57]:
# Create data generators
train_generator = train_datagen.flow(np.array(X_train), y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(np.array(X_val), y_val, batch_size=batch_size)
test_generator = test_datagen.flow(np.array(X_test), y_test, batch_size=batch_size)

MemoryError: Unable to allocate 2.40 GiB for an array with shape (6553, 128, 128, 3) and data type float64

In [ ]:
# Create a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [ ]:
# # Compile the model
# model.compile(loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])

In [ ]:
# Convert lists to NumPy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Check the shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# num_epochs = 10

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     batch_size=batch_size,
#     epochs=num_epochs,
#     validation_data=(X_val, y_val),
#     verbose=1
# )

In [ ]:
model_1= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(128, 128, 3)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_1.compile(loss= tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history = model_1.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model_1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot loss curves
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Plot loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()